In [1]:
from normalization.normalize import Normalize
from sklearn.datasets import fetch_openml



In [2]:

data = fetch_openml(name='credit-g')
denormalized_table = data.data.join(data.target)  # join on indexes
list(denormalized_table.columns.values)
norm = Normalize(denormalized_table=denormalized_table, denormalized_table_name="original_table")






/mnt/sda/boris_stanoev/PropStar/venv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:303: UserWarning: Multiple active versions of the dataset matching the name credit-g exist. Versions may be fundamentally different, returning version 1.
  warn(
/mnt/sda/boris_stanoev/PropStar/venv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
target_cols = ['class', 'checking_status', 'savings_status']
norm.create_entity(source_entity_name="original_table", target_entity_name="target", columns=target_cols,
                   distinct_values=True, retain_columns=["target_id"])
norm.entity_set['original_table']
norm.create_entity(source_entity_name="target", target_entity_name="checking_status_type", columns=['checking_status'],
                   distinct_values=True)
norm.create_entity(source_entity_name="target", target_entity_name="savings_status_type", columns=['savings_status'],
                   distinct_values=True)

In [4]:
credit_cols = ['duration', 'purpose', 'installment_commitment', 'credit_amount', "target_id"]
norm.create_entity(source_entity_name='original_table', target_entity_name='credit', columns=credit_cols,
                   distinct_values=False, retain_columns=["target_id"])
norm.add_custom_relation(parent_table_name="target", parent_table_column="target_id", child_table_name="credit",
                         child_table_column="target_id")
norm.entity_set.keys()
norm.create_entity(source_entity_name='credit', target_entity_name='credit_purpose_type', columns=['purpose'],
                   distinct_values=True)

In [5]:
person_cols = ['credit_history', 'employment', 'personal_status', 'other_parties', 'residence_since',
               'property_magnitude', 'age', 'other_payment_plans', 'housing', 'existing_credits', 'job',
               'num_dependents', 'own_telephone', 'foreign_worker', "target_id"]
norm.create_entity(source_entity_name='original_table', target_entity_name='person', columns=person_cols,
                   distinct_values=False, retain_columns=["target_id"])
norm.add_custom_relation(parent_table_name="target", parent_table_column="target_id", child_table_name="person",
                         child_table_column="target_id")
norm.create_entity(source_entity_name='person', target_entity_name='personal_status_type', columns=['personal_status'],
                   distinct_values=True)
norm.create_entity(source_entity_name='person', target_entity_name='employment_type', columns=['employment'],
                   distinct_values=True)
norm.create_entity(source_entity_name='person', target_entity_name='job_type', columns=['job'], distinct_values=True)
norm.create_entity(source_entity_name='person', target_entity_name='credit_history', columns=['credit_history'],
                   distinct_values=True)

In [6]:
property_cols = ['residence_since', 'property_magnitude', 'housing']
norm.create_entity(source_entity_name='person', target_entity_name='property', columns=property_cols,
                   distinct_values=True)


In [7]:
norm.remove_entity("original_table")


In [8]:

norm.primary_keys
norm.relations
norm.entity_set.keys()

dict_keys(['target', 'checking_status_type', 'savings_status_type', 'credit', 'credit_purpose_type', 'person', 'personal_status_type', 'employment_type', 'job_type', 'credit_history', 'property'])

In [9]:
for entity in norm.entity_set.keys():
    print(
        f"ALTER TABLE {entity} ADD CONSTRAINT PK_{entity} PRIMARY KEY ({norm.primary_keys[entity]});")

for parent_table in norm.relations:
    for child_table in norm.relations[parent_table]:
        print(f"ALTER TABLE {child_table} ADD CONSTRAINT FK_{child_table}_{parent_table} "
              f"FOREIGN KEY ({norm.relations[parent_table][child_table][1]}) "
              f"REFERENCES {parent_table}({norm.relations[parent_table][child_table][0]}));")



ALTER TABLE target ADD CONSTRAINT PK_target PRIMARY KEY (target_id);
ALTER TABLE checking_status_type ADD CONSTRAINT PK_checking_status_type PRIMARY KEY (checking_status_type_id);
ALTER TABLE savings_status_type ADD CONSTRAINT PK_savings_status_type PRIMARY KEY (savings_status_type_id);
ALTER TABLE credit ADD CONSTRAINT PK_credit PRIMARY KEY (credit_id);
ALTER TABLE credit_purpose_type ADD CONSTRAINT PK_credit_purpose_type PRIMARY KEY (credit_purpose_type_id);
ALTER TABLE person ADD CONSTRAINT PK_person PRIMARY KEY (person_id);
ALTER TABLE personal_status_type ADD CONSTRAINT PK_personal_status_type PRIMARY KEY (personal_status_type_id);
ALTER TABLE employment_type ADD CONSTRAINT PK_employment_type PRIMARY KEY (employment_type_id);
ALTER TABLE job_type ADD CONSTRAINT PK_job_type PRIMARY KEY (job_type_id);
ALTER TABLE credit_history ADD CONSTRAINT PK_credit_history PRIMARY KEY (credit_history_id);
ALTER TABLE property ADD CONSTRAINT PK_property PRIMARY KEY (property_id);
ALTER TABLE cred

In [ ]:
norm.persist_entity_set(database_name="credit_exp1")
